In [2]:
import numpy as np
import pandas as pd
import datetime

def parse_4096_bytes(data):
    """
    Parse the 4096-byte block following the logic provided in the script by Evgeny.
    """
    BASE_timestamp_ms = np.frombuffer(data[:4], np.uint32)[0]
    total_events = data[16] + (data[17] << 8)
    total_rejected = data[18] + (data[19] << 8)
    
    events = np.frombuffer(data[20:20 + 800 * 5], dtype=np.uint8).reshape(800, 5)
    
    flux_x200ms_F = np.frombuffer(data[4020:4020 + 23 * 2], np.uint16)
    flux_x200ms_H = np.frombuffer(data[4020 + 23 * 2:4020 + 23 * 3], np.uint8)
    flux_total = data[4096 - 5]
    DR_timestamp_us = np.frombuffer(data[4096 - 4:4096], np.uint32)[0]
    
    parsed_events = []
    for i in range(total_events):
        bt02 = (events[i][0] << 16) + (events[i][1] << 8) + events[i][2]
        bt34 = (events[i][3] << 8) + events[i][4]
        
        ts_us = bt02
        ts_ns = (bt34 >> 14) & 0b11
        ch = (bt34 >> 13) & 0b1
        h1 = bt34 & 0b1111111111111
        
        parsed_events.append({
            "BASE_timestamp_ms": BASE_timestamp_ms,
            "timestamp_us": ts_us,
            "channel": ch,
            "height": h1
        })
    
    return parsed_events

def read_bin_file_to_dataframe(filename, output_csv):
    packet_size = 4104
    timestamp_size = 8
    data_size = 4096
    
    all_events = []

    try:
        with open(filename, "rb") as file:
            while True:
                packet = file.read(packet_size)
                if len(packet) < packet_size:
                    break
                
                # Extract timestamp
                timestamp_ns = np.frombuffer(packet[:timestamp_size], dtype='datetime64[ns]')[0]
                #print(timestamp_ns)
                #dt = datetime.datetime.fromtimestamp(timestamp_ns.astype(int) / 1e9)
                global_timestamp = np.datetime64(timestamp_ns, 'ns')
                #print(global_timestamp)
                
                
                # Extract and parse board data
                board_data = packet[timestamp_size:]
                parsed_events = parse_4096_bytes(board_data)
                
                # Add global timestamp to each event and store in list
                for event in parsed_events:
                    event["global_timestamp"] = global_timestamp
                    all_events.append(event)
        
        # Create DataFrame
        df = pd.DataFrame(all_events)
        
        # Save to CSV
        df.to_csv(output_csv, index=False)
        print(f"Data successfully saved to {output_csv}")
        
        return df
        
    except FileNotFoundError:
        print(f"File {filename} not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example Usage
filename = '2024-12-17_14-56-59_RawDataWithTime.bin'  # Replace with your binary file path
output_csv = 'events_test_feb2025.csv'  # Replace with your desired CSV file name
df = read_bin_file_to_dataframe(filename, output_csv)

# Optional: Display the first few rows of the DataFrame
print(df.head())


Data successfully saved to events_test_feb2025.csv
   BASE_timestamp_ms  timestamp_us  channel  height  \
0              35169         18237        0     134   
1              35169         23255        0     188   
2              35169         25557        0     303   
3              35169         26062        0     381   
4              35169         26463        0     130   

               global_timestamp  
0 2024-12-17 13:56:58.988110480  
1 2024-12-17 13:56:58.988110480  
2 2024-12-17 13:56:58.988110480  
3 2024-12-17 13:56:58.988110480  
4 2024-12-17 13:56:58.988110480  


In [7]:
import numpy as np
import pandas as pd

# Define the same structured dtype used for writing
structured_dtype = np.dtype([('timestamp', np.uint64), ('height', np.uint16)])

# Read the binary file
with open("2025-04-02_11-00-51_Data.bin", "rb") as f:
    events_loaded = np.fromfile(f, dtype=structured_dtype)
    
# Convert to Pandas DataFrame
df = pd.DataFrame(events_loaded)

# Display the first few rows
print(df.head, df.tail)


<bound method NDFrame.head of        timestamp  height
0    32451760487     250
1    32451769170     144
2    32451790634     232
3    32451846520     178
4    32451864279     102
..           ...     ...
398  32455720638     186
399  32455735182     354
400  32455738936     157
401  32455745363     162
402  32455754196     208

[403 rows x 2 columns]> <bound method NDFrame.tail of        timestamp  height
0    32451760487     250
1    32451769170     144
2    32451790634     232
3    32451846520     178
4    32451864279     102
..           ...     ...
398  32455720638     186
399  32455735182     354
400  32455738936     157
401  32455745363     162
402  32455754196     208

[403 rows x 2 columns]>


In [9]:
import time
MICROS_PER_DAY = 24 * 60 * 60 * 1_000_000
print(time.time())
current_micros = 32455754196
print(current_micros)

# Convert to hours, minutes, seconds, and microseconds
hours = current_micros // 3_600_000_000
minutes = (current_micros % 3_600_000_000) // 60_000_000
seconds = (current_micros % 60_000_000) // 1_000_000
microseconds = current_micros % 1_000_000

# Format the output
formatted_time = f"{hours:02}:{minutes:02}:{seconds:02}.{microseconds:06}"
print(formatted_time)

1743584629.4210405
32455754196
09:00:55.754196
